# Overlap と mp_term_name の統合

-   例：abnormal_circulating_glucose_level（連続値）
-   例：male_infertility（カテゴリ値）


In [1]:
# Move up to top directory
import os
from pathlib import Path

print(os.getcwd())

while not Path("LICENSE").exists():
    os.chdir('../')

print(os.getcwd())

/mnt/e/Research/TSUMUGI/notebooks/notebools-web
/mnt/e/Research/TSUMUGI


In [2]:
P = print
from pprint import pprint as PP
from collections import Counter as C
from pathlib import Path
from collections import defaultdict
from itertools import combinations
import csv
import json
import pickle
import gzip
import shutil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 一括処理


In [3]:
overlap = pickle.load(open("data/overlap/overlapped_ratios_filtered.pkl", "rb"))
df_overlap = pd.DataFrame(
    overlap, columns=["marker1", "marker2", "overlap_ratio", "overlapped_mp_number", "overlapped_mp"]
)
df_overlap  # 240358 rows × 5 columns

,marker1,marker2,overlap_ratio,overlapped_mp_number,overlapped_mp
0,0610010K14Rik,4933427D14Rik,0.500,3,[embryonic lethality prior to organogenesis (H...
1,0610010K14Rik,Acvr1,0.600,3,[embryonic lethality prior to organogenesis (H...
2,0610010K14Rik,Adss2,0.375,3,[embryonic lethality prior to organogenesis (H...
3,0610010K14Rik,Ahcy,0.500,3,[embryonic lethality prior to organogenesis (H...
4,0610010K14Rik,Anapc4,0.750,3,[embryonic lethality prior to organogenesis (H...
...,...,...,...,...,...
134875,Zfp532,Zmynd8,0.273,3,"[decreased mean corpuscular volume (Hetero), p..."
134876,Zfp593,Zfp746,0.333,4,"[decreased bone mineral content (Homo), decrea..."
134877,Zfp638,Znhit2,0.375,3,[embryonic lethality prior to organogenesis (H...
134878,Zfp871,Zwilch,0.250,3,"[abnormal embryo size (Homo), embryonic growth..."


In [7]:
marker_mp = json.load(open("data/annotation/symbol_mptermname.json"))
marker_mp = pd.DataFrame(marker_mp.items(), columns=["marker_symbol", "mp_term_name"])
marker_mp # 7626 rows × 2 columns

,marker_symbol,mp_term_name
0,0610010K14Rik,[embryonic lethality prior to organogenesis (H...
1,0610040J01Rik,"[abnormal heart morphology (Homo), abnormal sp..."
2,1110059G10Rik,[vertebral transformation (Homo)]
3,1500009L16Rik,"[decreased bone mineral density (Homo), increa..."
4,1600014C10Rik,"[abnormal coat/hair pigmentation (Homo), abnor..."
...,...,...
7621,Zwint,[embryonic lethality prior to organogenesis (H...
7622,Zyg11b,[decreased exploration in new environment (Het...
7623,Zzef1,"[abnormal coat/hair pigmentation (Homo), abnor..."
7624,Zzz3,"[abnormal cornea morphology (Hetero), corneal ..."


In [8]:
print(len(df_overlap)) # 184250 → 134880
print(len(marker_mp)) # 7626

134880
7626


In [9]:
import shutil
output_dir = Path("data/network/mp_term_name")
# remove network directory
if output_dir.exists():
    shutil.rmtree(output_dir)

output_dir.mkdir(exist_ok=True, parents=True)


In [10]:
path_mp_terms = list(Path("data", "mp_term_name").glob("*.csv"))
# print(path_mp_terms[:3])
# print(len(path_mp_terms))
# path_mp_term = Path("data", "mp_term_name", "decreased_circulating_glucose_level.csv")

for path_mp_term in path_mp_terms:
    mp_term = path_mp_term.stem
    # print(mp_term)

    df_marker_effect = pd.read_csv(path_mp_term)
    df_marker_effect = df_marker_effect[["marker_symbol", "effect_size"]].drop_duplicates()
    df_marker_effect = df_marker_effect.dropna(subset=["effect_size"])

    # Absolute value of effect size
    df_marker_effect.loc[:, "effect_size"] = df_marker_effect["effect_size"].abs()

    df_filtered = df_overlap[
        (df_overlap['marker1'].isin(df_marker_effect['marker_symbol'])) &
        (df_overlap['marker2'].isin(df_marker_effect['marker_symbol']))
    ]
    ## 出力
    ### Nodeを作成する
    df_marker1 = df_filtered[["marker1"]]
    df_marker2 = df_filtered[["marker2"]]
    df_node_marker1 = pd.merge(df_marker1, df_marker_effect, left_on='marker1', right_on='marker_symbol', how='inner')[["marker_symbol"]]
    df_node_marker2 = pd.merge(df_marker2, df_marker_effect, left_on='marker2', right_on='marker_symbol', how='inner')[["marker_symbol"]]

    df_node = pd.concat([df_node_marker1, df_node_marker2], axis=0).drop_duplicates()


    df_node = pd.merge(df_node, marker_mp, how='inner', on='marker_symbol')
    df_node = pd.merge(df_node, df_marker_effect, how='inner', on='marker_symbol')


    # NodeをJSON形式に変換
    node_json = []
    for _, row in df_node.iterrows():
        node_json.append({
            "data": {
                "id": row['marker_symbol'],
                "label": row['marker_symbol'],
                "annotation": row['mp_term_name'],
                "node_color": row['effect_size']
            }
        })

    ### Edgeを作成する
    df_edge = df_filtered[["marker1", "marker2", "overlap_ratio", "overlapped_mp"]]
    # EdgeをJSON形式に変換
    edge_json = []
    for _, row in df_edge.iterrows():
        edge_json.append({
            "data": {
                "source": row['marker1'],
                "target": row['marker2'],
                "annotation": row['overlapped_mp'],
                "edge_size": row['overlap_ratio']
            }
        })

    ### EdgeとNodeを統合して、出力
    # Combine node and edge

    network_json = node_json + edge_json

    # Output as JSON
    if network_json:
        output_json = output_dir / f"{mp_term}.json.gz"
        with gzip.open(output_json, "wt", encoding="utf-8") as f:
            json.dump(network_json, f, indent=2)

# 39.5 seconds

In [11]:
%%bash

ls -lh data/network/mp_term_name/ | head -n 5 # 9.0M

total 9.0M
-rwxrwxrwx 1 kuno kuno  933 Jan 11 07:28 abnormal_QT_variability.json.gz
-rwxrwxrwx 1 kuno kuno  741 Jan 11 07:28 abnormal_abdominal_wall_morphology.json.gz
-rwxrwxrwx 1 kuno kuno 2.1K Jan 11 07:28 abnormal_adrenal_gland_morphology.json.gz
-rwxrwxrwx 1 kuno kuno 2.9K Jan 11 07:28 abnormal_allantois_morphology.json.gz


# Debug


In [ ]:
# path_overlap = Path("data", "overlap_ratios_filtered.csv")
# df_overlap = pd.read_csv(path_overlap)

overlap = pickle.load(open("data/overlapped_ratios_filtered.pkl", "rb"))
df_overlap = pd.DataFrame(overlap, columns=["marker1", "marker2", "overlap_ratio", "overlapped_mp_number", "overlapped_mp"])
P(len(df_overlap)) # 184250
df_overlap.head(3)

184250


,marker1,marker2,overlap_ratio,overlapped_mp_number,overlapped_mp
0,Rhd,Ap4e1,0.071,5,"[decreased circulating free fatty acids level,..."
1,Rhd,Asxl1,0.136,3,"[decreased hemoglobin content, decreased mean ..."
2,Rhd,Abcg8,0.091,4,"[decreased hemoglobin content, decreased mean ..."


In [ ]:
path_mp_terms = list(Path("data", "mp_term_name").glob("*.csv"))
print(path_mp_terms[:3])
print(len(path_mp_terms)) # 670

[PosixPath('data/mp_term_name/abnormal_abdominal_wall_morphology.csv'), PosixPath('data/mp_term_name/abnormal_adipose_tissue_amount.csv'), PosixPath('data/mp_term_name/abnormal_adrenal_gland_morphology.csv')]
670


In [ ]:
# path_mp_term = Path("data", "mp_term_name", "decreased_circulating_glucose_level.csv")
path_mp_term = Path("data", "mp_term_name", "increased_circulating_glucose_level.csv")
mp_term = path_mp_term.stem
print(mp_term)


increased_circulating_glucose_level


In [ ]:
df_marker_effect = pd.read_csv(path_mp_term)
df_marker_effect = df_marker_effect[["marker_symbol", "effect_size"]].drop_duplicates()

# effect_sizeがNanの行を削除
df_marker_effect = df_marker_effect.dropna(subset=["effect_size"])

print(len(df_marker_effect))

df_marker_effect.head(3)


73


,marker_symbol,effect_size
0,Zfp513,0.200596
1,Dbnl,0.828539
2,Oas1g,0.922229


In [ ]:
# print(df_mp["effect_size"].describe())

In [ ]:
# Absolute value of effect size

df_marker_effect.loc[:, "effect_size"] = df_marker_effect["effect_size"].abs()

In [ ]:
# print(df_overlap[df_overlap["intersect_count"] > 5 & df_overlap["overlap_ratio"] > 0.5])

In [ ]:
# df_overlap_filterd = df_overlap[df_overlap["overlapped_mp_number"] > 0]
# df_overlap_filterd = df_overlap[(df_overlap["overlap_ratio"] > 0.25)]
df_overlap_filterd = df_overlap

In [ ]:
df_filtered = df_overlap_filterd[
    (df_overlap_filterd['marker1'].isin(df_marker_effect['marker_symbol'])) &
    (df_overlap_filterd['marker2'].isin(df_marker_effect['marker_symbol']))
]

In [ ]:

print(len(df_filtered))

df_filtered.head(3)

213


,marker1,marker2,overlap_ratio,overlapped_mp_number,overlapped_mp
2704,Zfyve19,Rnf10,0.079,5,"[abnormal behavior, decreased exploration in n..."
2712,Zfyve19,Usp31,0.136,3,"[decreased exploration in new environment, dec..."
2722,Zfyve19,Ky,0.273,3,"[abnormal behavior, decreased exploration in n..."


## 出力


### Node を作成する


In [ ]:
# merge mp term
df_marker1 = df_filtered[["marker1"]]
df_marker2 = df_filtered[["marker2"]]
df_node_marker1 = pd.merge(df_marker1, df_marker_effect, left_on='marker1', right_on='marker_symbol', how='inner')[["marker_symbol"]]
df_node_marker2 = pd.merge(df_marker2, df_marker_effect, left_on='marker2', right_on='marker_symbol', how='inner')[["marker_symbol"]]

df_node = pd.concat([df_node_marker1, df_node_marker2], axis=0).drop_duplicates()

import pickle
marker_mp = pickle.load(open("data/marker_mp.pkl", "rb"))
marker_mp = pd.DataFrame(marker_mp.items(), columns=["marker_symbol", "mp_term"])

df_node = pd.merge(df_node, marker_mp, how='inner', on='marker_symbol')
df_node = pd.merge(df_node, df_marker_effect, how='inner', on='marker_symbol')

df_node[:3]


,marker_symbol,mp_term,effect_size
0,Zfyve19,"[abnormal behavior, decreased exploration in n...",0.695784
1,Csnk1g2,"[abnormal auditory brainstem response, abnorma...",0.665518
2,Bbs5,"[abnormal bone structure, abnormal gait, abnor...",1.902677


In [ ]:
# JSON形式に変換
node_json = []
for _, row in df_node.iterrows():
    node_json.append({
        "data": {
            "id": row['marker_symbol'],
            "label": row['marker_symbol'],
            "annotation": row['mp_term'],
            "node_color": row['effect_size']
        }
    })

print(node_json[:3])

[{'data': {'id': 'Zfyve19', 'label': 'Zfyve19', 'annotation': ['abnormal behavior', 'decreased exploration in new environment', 'decreased locomotor activity', 'decreased thigmotaxis', 'increased circulating glucose level'], 'node_color': 0.695783774710263}}, {'data': {'id': 'Csnk1g2', 'label': 'Csnk1g2', 'annotation': ['abnormal auditory brainstem response', 'abnormal lens morphology', 'abnormal retina blood vessel morphology', 'abnormal retina morphology', 'abnormal retina vasculature morphology', 'decreased total retina thickness', 'increased cardiac muscle contractility', 'increased circulating aspartate transaminase level', 'increased circulating glucose level', 'increased circulating phosphate level', 'increased circulating potassium level', 'increased heart rate variability', 'increased mean corpuscular volume', 'increased pulmonary respiratory rate', 'persistence of hyaloid vascular system'], 'node_color': 0.665518283420352}}, {'data': {'id': 'Bbs5', 'label': 'Bbs5', 'annotatio

### Edge を作成する


In [ ]:
df_edge = df_filtered[["marker1", "marker2", "overlap_ratio", "overlapped_mp"]]
print(len(df_edge))
df_edge.head(3)

213


,marker1,marker2,overlap_ratio,overlapped_mp
2704,Zfyve19,Rnf10,0.079,"[abnormal behavior, decreased exploration in n..."
2712,Zfyve19,Usp31,0.136,"[decreased exploration in new environment, dec..."
2722,Zfyve19,Ky,0.273,"[abnormal behavior, decreased exploration in n..."


In [ ]:
# JSON形式に変換
edge_json = []
for _, row in df_edge.iterrows():
    edge_json.append({
        "data": {
            "source": row['marker1'],
            "target": row['marker2'],
            "annotation": row['overlapped_mp'],
            "edge_size": row['overlap_ratio']
        }
    })

print(edge_json[:3])

[{'data': {'source': 'Zfyve19', 'target': 'Rnf10', 'annotation': ['abnormal behavior', 'decreased exploration in new environment', 'decreased locomotor activity', 'decreased thigmotaxis', 'increased circulating glucose level'], 'edge_size': 0.079}}, {'data': {'source': 'Zfyve19', 'target': 'Usp31', 'annotation': ['decreased exploration in new environment', 'decreased locomotor activity', 'increased circulating glucose level'], 'edge_size': 0.136}}, {'data': {'source': 'Zfyve19', 'target': 'Ky', 'annotation': ['abnormal behavior', 'decreased exploration in new environment', 'increased circulating glucose level'], 'edge_size': 0.273}}]


### Edge と Node を統合して、出力


In [ ]:
# Combine node and edge

network_json = node_json + edge_json

In [ ]:
print(len(network_json))
print(network_json[:3])
print(network_json[-3:])

269
[{'data': {'id': 'Zfyve19', 'label': 'Zfyve19', 'annotation': ['abnormal behavior', 'decreased exploration in new environment', 'decreased locomotor activity', 'decreased thigmotaxis', 'increased circulating glucose level'], 'node_color': 0.695783774710263}}, {'data': {'id': 'Csnk1g2', 'label': 'Csnk1g2', 'annotation': ['abnormal auditory brainstem response', 'abnormal lens morphology', 'abnormal retina blood vessel morphology', 'abnormal retina morphology', 'abnormal retina vasculature morphology', 'decreased total retina thickness', 'increased cardiac muscle contractility', 'increased circulating aspartate transaminase level', 'increased circulating glucose level', 'increased circulating phosphate level', 'increased circulating potassium level', 'increased heart rate variability', 'increased mean corpuscular volume', 'increased pulmonary respiratory rate', 'persistence of hyaloid vascular system'], 'node_color': 0.665518283420352}}, {'data': {'id': 'Bbs5', 'label': 'Bbs5', 'annot

In [ ]:
# Output as JSON file
Path("data/network").mkdir(exist_ok=True, parents=True)
with open(f"data/network/{mp_term}.json", "w") as f:
    json.dump(network_json, f, indent=2)

In [32]:
marker_mp

,marker_symbol,mp_term
0,Dpf2,"[Abnormal craniofacial morphology, Abnormal em..."
1,Gna13,"[Abnormal embryo size, Abnormal visceral yolk ..."
2,Dynlrb2,"[Increased fasting circulating glucose level, ..."
3,Slc16a2,"[Decreased circulating cholesterol level, Decr..."
4,Galr1,"[Decreased lymphocyte cell number, Enlarged sp..."
...,...,...
7167,Adgrb1,[Increased circulating insulin level]
7168,Amigo3,[Enlarged heart]
7169,Hap1,"[Preweaning lethality, complete penetrance]"
7170,Slc23a3,[Abnormal lung morphology]


In [38]:
for data in node_json:
    if "Lepr" == data["data"]["id"]:
        print(data)

{'data': {'id': 'Lepr', 'label': 'Lepr', 'annotation': ['Abnormal bone structure', 'Abnormal gait', 'Abnormal locomotor behavior', 'Decreased bone mineral content', 'Decreased bone mineral density', 'Decreased circulating chloride level', 'Decreased exploration in new environment', 'Decreased heart weight', 'Decreased lean body mass', 'Decreased leukocyte cell number', 'Decreased locomotor activity', 'Decreased lymphocyte cell number', 'Decreased spleen weight', 'Enlarged liver', 'Female infertility', 'Impaired righting response', 'Increased anxiety-related response', 'Increased circulating alanine transaminase level', 'Increased circulating alkaline phosphatase level', 'Increased circulating aspartate transaminase level', 'Increased circulating calcium level', 'Increased circulating cholesterol level', 'Increased circulating fructosamine level', 'Increased circulating glucose level', 'Increased circulating hdl cholesterol level', 'Increased circulating iron level', 'Increased circulat

In [163]:
for data in edge_json:
    if "Herc1" == data["data"]["source"]:
        print(data)
    if "Herc1" == data["data"]["target"]:
        print(data)

{'data': {'source': 'Camsap3', 'target': 'Herc1', 'annotation': ['abnormal auditory brainstem response', 'abnormal lens morphology', 'cataract', 'decreased bone mineral content', 'decreased bone mineral density', 'decreased lean body mass', 'increased leukocyte cell number', 'increased total body fat amount', 'male infertility'], 'edge_size': 0.31}}
{'data': {'source': 'Prdm14', 'target': 'Herc1', 'annotation': ['decreased bone mineral content', 'decreased bone mineral density', 'decreased lean body mass', 'impaired glucose tolerance', 'increased total body fat amount', 'male infertility'], 'edge_size': 0.273}}
{'data': {'source': 'Kash5', 'target': 'Herc1', 'annotation': ['decreased bone mineral content', 'decreased bone mineral density', 'decreased lean body mass', 'increased leukocyte cell number', 'increased total body fat amount', 'male infertility'], 'edge_size': 0.286}}
{'data': {'source': 'Trim37', 'target': 'Herc1', 'annotation': ['abnormal auditory brainstem response', 'abnor

In [150]:
for data in edge_json:
    if "Stoml2" == data["data"]["source"]:
        print(data)
    if "Stoml2" == data["data"]["target"]:
        print(data)

{'data': {'source': 'Vgf', 'target': 'Stoml2', 'annotation': ['decreased prepulse inhibition', 'increased circulating alanine transaminase level', 'increased circulating aspartate transaminase level', 'male infertility'], 'edge_size': 0.25}}
{'data': {'source': 'Fbxo38', 'target': 'Stoml2', 'annotation': ['increased circulating alanine transaminase level', 'increased circulating aspartate transaminase level', 'male infertility'], 'edge_size': 0.214}}
{'data': {'source': 'Stoml2', 'target': 'Clcn1', 'annotation': ['decreased prepulse inhibition', 'increased circulating aspartate transaminase level', 'increased circulating bilirubin level', 'male infertility'], 'edge_size': 0.182}}
{'data': {'source': 'Stoml2', 'target': 'Lepr', 'annotation': ['increased circulating alanine transaminase level', 'increased circulating aspartate transaminase level', 'male infertility'], 'edge_size': 0.073}}
{'data': {'source': 'Stoml2', 'target': 'Atp13a3', 'annotation': ['increased circulating alanine tra